# Unidad 03

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [ ]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

## Usando variables

In [ ]:
USE Pampero;
DECLARE @maxid AS INT = (SELECT MAX(IDPedido) FROM Pedidos);
SELECT IDPedido, FechaPedido, IDEmpleado, IDCliente
FROM Pedidos
WHERE IDPedido = @maxid;

## Usando Subconsultas

In [ ]:
SELECT IDPedido, FechaPedido, IDEmpleado, IDCliente
FROM Pedidos
WHERE IDPedido = (SELECT MAX(O.IDPedido)
FROM Pedidos AS O);

## Comparando por =

In [ ]:
SELECT IDPedido
FROM Pedidos
WHERE IDEmpleado =
(SELECT E.IDEmpleado
FROM Empleados AS E
WHERE E.Apellido LIKE 'C%');

In [ ]:
SELECT IDPedido
FROM Pedidos
WHERE IDEmpleado =
(SELECT E.IDEmpleado
FROM Empleados AS E
WHERE E.Apellido LIKE 'D%');

## Usando IN

In [ ]:
SELECT IDPedido
FROM Pedidos
WHERE IDEmpleado IN
(SELECT E.IDEmpleado
FROM Empleados AS E
WHERE E.Apellido LIKE 'D%');

## Subconsulta correlacionada

In [ ]:
SELECT IDCliente, IDPedido, FechaPedido, IDEmpleado
FROM Pedidos AS P1
WHERE IDPedido =
(SELECT MAX(P2.IDPedido)
FROM Pedidos AS P2
WHERE P2.IDCliente = P1.IDCliente);

## Exists

In [ ]:
SELECT IDCliente, NombreEmpresa
FROM Clientes AS C
WHERE Pais = 'España'
AND EXISTS
(SELECT * FROM Pedidos AS P
WHERE P.IDCliente = C.IDCliente);

In [ ]:
SELECT IDCliente, NombreEmpresa
FROM Clientes AS C
WHERE pais = 'España'
AND NOT EXISTS
(SELECT * FROM Pedidos AS P
WHERE P.IDCliente = C.IDCliente);

## Con NOT IN

In [ ]:
SELECT IDCliente, NombreEmpresa
FROM Clientes AS C
WHERE pais = 'España'
AND IDCliente NOT IN
(SELECT IDCliente FROM Pedidos AS P WHERE IDCliente IS NOT NULL);

## Si agrego un Pedido...

In [ ]:
INSERT INTO [Pedidos]
([IDCliente], [IDEmpleado], [FechaPedido], [FechaRequerida], [FechaEnvio], [EnvioPor], [Flete],
[NombreEnvio], [DireccionEnvio], [CiudadEnvio], [RegionEnvio], [CodigoPostalEnvio], [PaisEnvio])
VALUES
(NULL, 1, '20090212', '20090212', '20090212', 1, 123.00, 
'abc', 'abc', 'abc', 'abc', 'abc', 'abc')

## Chequeando NOT NULL

In [ ]:
SELECT IDCliente, NombreEmpresa
FROM Clientes AS C
WHERE pais = 'España'
AND IDCliente NOT IN
(SELECT IDCliente FROM Pedidos AS P
WHERE IDCliente IS NOT NULL);

In [ ]:
DELETE FROM Pedidos WHERE IDCliente IS NULL;

## Division Relacional

In [ ]:
SELECT IDEmpleado
FROM Empleados
WHERE Pais = 'EE.UU.'

In [ ]:
SELECT IDCliente 
FROM Pedidos
WHERE IDEmpleado IN (1, 2, 3, 4, 8)
GROUP BY IDCliente
HAVING COUNT(DISTINCT IDEmpleado) = 5;

In [ ]:
SELECT IDCliente 
FROM Pedidos
WHERE IDEmpleado IN 
(SELECT IDEmpleado 
FROM Empleados 
WHERE Pais = 'EE.UU.')
GROUP BY IDCliente
HAVING COUNT(DISTINCT IDEmpleado) = 
(SELECT COUNT(*) 
FROM Empleados
WHERE Pais = 'EE.UU.');

In [ ]:
SELECT IDCliente FROM Clientes AS C
WHERE NOT EXISTS
(SELECT * FROM Empleados AS E
WHERE pais = 'EE.UU.'
AND NOT EXISTS
(SELECT * FROM Pedidos AS P
WHERE P.IDCliente = C.IDCliente
AND P.IDEmpleado = E.IDEmpleado));